In [ ]:
-- Trigger criada pelo Activesoft Service Pack em 10/06/2019
-- durante atualização para versão 1.064.907.
CREATE TRIGGER dbo.TgAudit_TbDiarioAlunoI ON dbo.TbDiarioAluno FOR INSERT AS 
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiarioAluno' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 5 
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiarioAluno inválida)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	IF ISNULL( (	SELECT	SUM( CASE WHEN c.COLUMN_NAME IN ( '','IdDiarioAluno' ) THEN 1 ELSE 100 END )
			FROM	INFORMATION_SCHEMA.TABLE_CONSTRAINTS pk, 
				INFORMATION_SCHEMA.KEY_COLUMN_USAGE c 
			WHERE	CONSTRAINT_TYPE = 'PRIMARY KEY'
				AND c.TABLE_NAME = pk.TABLE_NAME
				AND c.CONSTRAINT_NAME = pk.CONSTRAINT_NAME
				AND pk.TABLE_NAME = 'TbDiarioAluno' ), 0 ) <> 1
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiarioAluno inválida -- definição da chave primária)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable') IS NULL 
	  BEGIN 
		--RAISERROR ('Dados não podem ser alterados (autenticação)', 16, 1) 
		--ROLLBACK TRANSACTION 
		--RETURN 
		SET @IdUsuario = NULL
	  END
	ELSE
	  SELECT @IdUsuario = IdUsuario FROM #TempTable



	DECLARE	@Descricao VARCHAR(8000), 
		@Value VARCHAR(500) 

	DECLARE xAudit CURSOR FOR 
		SELECT  IdDiarioAluno, IdDiario, IdAluno, DataInicial, DataFinal 
		FROM	inserted

	OPEN xAudit
  
	DECLARE @Audit_IdDiarioAluno int, @Audit_IdDiario int, @Audit_IdAluno int, @Audit_DataInicial smalldatetime, @Audit_DataFinal smalldatetime
  
	FETCH NEXT FROM xAudit INTO @Audit_IdDiarioAluno, @Audit_IdDiario, @Audit_IdAluno, @Audit_DataInicial, @Audit_DataFinal
	WHILE @@FETCH_STATUS = 0
	  BEGIN
		SET @Descricao = ''


		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_IdDiarioAluno ), '<nulo>' )
		SET @Descricao = @Descricao + 'IdDiarioAluno: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT Descricao FROM TbDiario WHERE IdDiario = @Audit_IdDiario ), '<nulo>' )
		SET @Descricao = @Descricao + 'IdDiario: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT NomeAluno FROM TbAluno WHERE IdAluno = @Audit_IdAluno ), '<nulo>' )
		SET @Descricao = @Descricao + 'IdAluno: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataInicial, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataInicial, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'DataInicial: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataFinal, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataFinal, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'DataFinal: ' + @Value + CHAR(13)

		INSERT TbAuditoria ( Tabela, IdUsuario, ValorChave1, ValorChave2, ValorChave3, ValorChave4, ValorChave5, Operacao, DescricaoOperacao )
			SELECT 'TbDiarioAluno', @IdUsuario, @Audit_IdDiarioAluno, @Audit_IdDiario, @Audit_IdAluno, NULL, NULL, 'I', @Descricao

		FETCH NEXT FROM xAudit INTO @Audit_IdDiarioAluno, @Audit_IdDiario, @Audit_IdAluno, @Audit_DataInicial, @Audit_DataFinal
	  END

	CLOSE xAudit
	DEALLOCATE xAudit
END

In [ ]:
-- Trigger criada pelo Activesoft Service Pack em 10/06/2019
-- durante atualização para versão 1.064.907.
CREATE TRIGGER dbo.TgAudit_TbDiarioAlunoU ON dbo.TbDiarioAluno FOR UPDATE AS 
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiarioAluno' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 5 
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiarioAluno inválida -- quantidade de campos)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	IF ISNULL( (	SELECT	SUM( CASE WHEN c.COLUMN_NAME IN ( '','IdDiarioAluno' ) THEN 1 ELSE 100 END )
			FROM	INFORMATION_SCHEMA.TABLE_CONSTRAINTS pk, 
				INFORMATION_SCHEMA.KEY_COLUMN_USAGE c 
			WHERE	CONSTRAINT_TYPE = 'PRIMARY KEY'
				AND c.TABLE_NAME = pk.TABLE_NAME
				AND c.CONSTRAINT_NAME = pk.CONSTRAINT_NAME
				AND pk.TABLE_NAME = 'TbDiarioAluno' ), 0 ) <> 1
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiarioAluno inválida -- definição da chave primária)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 

	  
	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable') IS NULL 
	  BEGIN 
		--RAISERROR ('Dados não podem ser alterados (autenticação)', 16, 1) 
		--ROLLBACK TRANSACTION 
		--RETURN 
		SET @IdUsuario = NULL
	  END
	ELSE
	  SELECT @IdUsuario = IdUsuario FROM #TempTable


	DECLARE	@Descricao VARCHAR(8000), 
		@StHouveCampoAlterado BIT,
		@OldValue VARCHAR(500), 
		@NewValue VARCHAR(500) 

	DECLARE xAudit CURSOR FOR 
		SELECT  d.IdDiarioAluno, i.IdDiarioAluno, d.IdDiario, i.IdDiario, d.IdAluno, i.IdAluno, d.DataInicial, i.DataInicial, d.DataFinal, i.DataFinal 
		FROM	deleted d 
		FULL	OUTER JOIN inserted i ON d.IdDiarioAluno  =  i.IdDiarioAluno

	OPEN xAudit 
  
	DECLARE @OLD_IdDiarioAluno int, @NEW_IdDiarioAluno int, @OLD_IdDiario int, @NEW_IdDiario int, @OLD_IdAluno int, @NEW_IdAluno int, @OLD_DataInicial smalldatetime, @NEW_DataInicial smalldatetime, @OLD_DataFinal smalldatetime, @NEW_DataFinal smalldatetime
  
	FETCH NEXT FROM xAudit INTO @OLD_IdDiarioAluno, @NEW_IdDiarioAluno, @OLD_IdDiario, @NEW_IdDiario, @OLD_IdAluno, @NEW_IdAluno, @OLD_DataInicial, @NEW_DataInicial, @OLD_DataFinal, @NEW_DataFinal
	WHILE @@FETCH_STATUS = 0
	  BEGIN
		SET @Descricao = ''
		SET @StHouveCampoAlterado = 0


		SET @OldValue = ISNULL( CONVERT( VARCHAR(500), @OLD_IdDiarioAluno ), '<nulo>' )
		SET @NewValue = ISNULL( CONVERT( VARCHAR(500), @NEW_IdDiarioAluno ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'IdDiarioAluno: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( ( SELECT Descricao FROM TbDiario WHERE IdDiario = @OLD_IdDiario ), '<nulo>' )
		SET @NewValue = ISNULL( ( SELECT Descricao FROM TbDiario WHERE IdDiario = @NEW_IdDiario ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'IdDiario: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( ( SELECT NomeAluno FROM TbAluno WHERE IdAluno = @OLD_IdAluno ), '<nulo>' )
		SET @NewValue = ISNULL( ( SELECT NomeAluno FROM TbAluno WHERE IdAluno = @NEW_IdAluno ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'IdAluno: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @OLD_DataInicial, 103 ) + ' ' + CONVERT( VARCHAR, @OLD_DataInicial, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @NewValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @NEW_DataInicial, 103 ) + ' ' + CONVERT( VARCHAR, @NEW_DataInicial, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'DataInicial: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @OLD_DataFinal, 103 ) + ' ' + CONVERT( VARCHAR, @OLD_DataFinal, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @NewValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @NEW_DataFinal, 103 ) + ' ' + CONVERT( VARCHAR, @NEW_DataFinal, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'DataFinal: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		IF ( @StHouveCampoAlterado = 1 ) AND ( @Descricao <> '' )
			INSERT TbAuditoria ( Tabela, IdUsuario, ValorChave1, ValorChave2, ValorChave3, ValorChave4, ValorChave5, Operacao, DescricaoOperacao )
				SELECT 'TbDiarioAluno', @IdUsuario, @NEW_IdDiarioAluno, @NEW_IdDiario, @NEW_IdAluno, NULL, NULL, 'U', @Descricao

		FETCH NEXT FROM xAudit INTO @OLD_IdDiarioAluno, @NEW_IdDiarioAluno, @OLD_IdDiario, @NEW_IdDiario, @OLD_IdAluno, @NEW_IdAluno, @OLD_DataInicial, @NEW_DataInicial, @OLD_DataFinal, @NEW_DataFinal
	  END


	CLOSE xAudit
	DEALLOCATE xAudit

END


In [ ]:
-- Trigger criada pelo Activesoft Service Pack em 10/06/2019
-- durante atualização para versão 1.064.907.
CREATE TRIGGER dbo.TgAudit_TbDiarioAlunoD ON dbo.TbDiarioAluno FOR DELETE AS 
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiarioAluno' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 5 
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiarioAluno inválida)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	IF ISNULL( (	SELECT	SUM( CASE WHEN c.COLUMN_NAME IN ( '','IdDiarioAluno' ) THEN 1 ELSE 100 END )
			FROM	INFORMATION_SCHEMA.TABLE_CONSTRAINTS pk, 
				INFORMATION_SCHEMA.KEY_COLUMN_USAGE c 
			WHERE	CONSTRAINT_TYPE = 'PRIMARY KEY'
				AND c.TABLE_NAME = pk.TABLE_NAME
				AND c.CONSTRAINT_NAME = pk.CONSTRAINT_NAME
				AND pk.TABLE_NAME = 'TbDiarioAluno' ), 0 ) <> 1
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiarioAluno inválida -- definição da chave primária)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 	  
	  
	  
	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable') IS NULL 
	  BEGIN 
		--RAISERROR ('Dados não podem ser alterados (autenticação)', 16, 1) 
		--ROLLBACK TRANSACTION 
		--RETURN 
		SET @IdUsuario = NULL
	  END
	ELSE
	  SELECT @IdUsuario = IdUsuario FROM #TempTable



	DECLARE	@Descricao VARCHAR(8000), 
		@Value VARCHAR(500) 

	DECLARE xAudit CURSOR FOR 
		SELECT  IdDiarioAluno, IdDiario, IdAluno, DataInicial, DataFinal 
		FROM	deleted

	OPEN xAudit
  
	DECLARE @Audit_IdDiarioAluno int, @Audit_IdDiario int, @Audit_IdAluno int, @Audit_DataInicial smalldatetime, @Audit_DataFinal smalldatetime
  
	FETCH NEXT FROM xAudit INTO @Audit_IdDiarioAluno, @Audit_IdDiario, @Audit_IdAluno, @Audit_DataInicial, @Audit_DataFinal
	WHILE @@FETCH_STATUS = 0
	  BEGIN
		SET @Descricao = ''


		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_IdDiarioAluno ), '<nulo>' )
		SET @Descricao = @Descricao + 'IdDiarioAluno: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT Descricao FROM TbDiario WHERE IdDiario = @Audit_IdDiario ), '<nulo>' )
		SET @Descricao = @Descricao + 'IdDiario: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT NomeAluno FROM TbAluno WHERE IdAluno = @Audit_IdAluno ), '<nulo>' )
		SET @Descricao = @Descricao + 'IdAluno: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataInicial, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataInicial, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'DataInicial: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataFinal, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataFinal, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'DataFinal: ' + @Value + CHAR(13)

		INSERT TbAuditoria ( Tabela, IdUsuario, ValorChave1, ValorChave2, ValorChave3, ValorChave4, ValorChave5, Operacao, DescricaoOperacao )
			SELECT 'TbDiarioAluno', @IdUsuario, @Audit_IdDiarioAluno, @Audit_IdDiario, @Audit_IdAluno, NULL, NULL, 'D', @Descricao

		FETCH NEXT FROM xAudit INTO @Audit_IdDiarioAluno, @Audit_IdDiario, @Audit_IdAluno, @Audit_DataInicial, @Audit_DataFinal
	  END

	CLOSE xAudit
	DEALLOCATE xAudit
END
